In [1]:
import os
import numpy as np
from dotenv import load_dotenv
from openai import AzureOpenAI
from pprint import pprint
from utils.prompts.prompt import *
from utils.prompts.Schema_Prompt_v1 import *
from utils.postgres_functions import *

In [2]:
load_dotenv()

True

In [3]:
def calculate_confidence_score(logprobs: list) -> float:
    probs = []
    # iterate through the token predictions and get the linear probs [0;1]
    for logprob in logprobs:
        probs.append(np.round(np.exp(logprob)*100, 2))
    # calculate the confidence
    return round(np.mean(probs), 2)

In [4]:
client = AzureOpenAI(
    api_key = str(os.environ['azure_api_key']),
    api_version = str(os.environ['azure_api_version']),
    azure_endpoint = str(os.environ['azure_api_endpoint'])
)

In [8]:
# QUERY = "Forgalomban van a novocetrin?"
# QUERY = "Milyen ciszplatin hatóanyagú gyógyszert tudok filmtablettás gyógyszerként kiírni?"
# QUERY = "Milyen ciszplatin hatóanyagú gyógyszert tudok kiírni?"
# QUERY = "Mennyibe kerül a syntroxine?"
QUERY = "Milyen gyógyszert lenne érdemes felírni láz esetén?"
# QUERY = "Milyen kiszerelésben kapható az egilok tabletta?"
# QUERY = "Ki forgalmazza a perdox oldatot?"
# QUERY = "Mivel lehet kiváltani a Matrifent?"
# QUERY = "Ki írhatja fel a letrozol pharcet?"
# QUERY = "Milyen betegségek esetén alkalmazható a Brusimp?"
# QUERY = "Mi a calcium fluoratum ATC kódjának értelmezése?"

In [9]:
sql_gen_messages = [
                {"role": "system", "content" : sql_system_message.format(system_prompt=SQL_SYSTEM_PROMPT, 
                                                                    table_info=SCHEMA_PROMPT)},
                {"role": "user", "content": question_message.format(question=QUERY)}
                ]

In [10]:
full_response = client.chat.completions.create(
    model="TextToSql_Pupha",
    messages=sql_gen_messages,
    temperature=0.0,
    max_tokens=1024,
    logprobs=True
)
pprint(full_response.choices[0].message.content)

('SELECT OEP_NEV, HATOANYAG FROM GYOGYSZ WHERE HATOANYAG ILIKE '
 "'%lázcsillapító%' OR HATOANYAG ILIKE '%paracetamol%' OR HATOANYAG ILIKE "
 "'%ibuprofen%' OR HATOANYAG ILIKE '%aszpirin%' LIMIT 10")


In [11]:
pprint(full_response)

ChatCompletion(id='chatcmpl-9G3qlZwgEqaIfvPe3bsLWlVPjBgBk', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='SELECT', bytes=[83, 69, 76, 69, 67, 84], logprob=-0.036547907, top_logprobs=[]), ChatCompletionTokenLogprob(token=' O', bytes=[32, 79], logprob=-0.49334645, top_logprobs=[]), ChatCompletionTokenLogprob(token='EP', bytes=[69, 80], logprob=-2.868329e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token='_NE', bytes=[95, 78, 69], logprob=-0.00025251842, top_logprobs=[]), ChatCompletionTokenLogprob(token='V', bytes=[86], logprob=0.0, top_logprobs=[]), ChatCompletionTokenLogprob(token=',', bytes=[44], logprob=-0.012823578, top_logprobs=[]), ChatCompletionTokenLogprob(token=' H', bytes=[32, 72], logprob=-0.09125859, top_logprobs=[]), ChatCompletionTokenLogprob(token='ATO', bytes=[65, 84, 79], logprob=-1.8431772e-05, top_logprobs=[]), ChatCompletionTokenLogprob(token='ANY', bytes=[65, 78, 89], logprob=-1.1994775e-05, to

In [9]:
logprobs = [x.logprob for x in full_response.choices[0].logprobs.content]
calculate_confidence_score(logprobs)

94.66

In [12]:
db_connection = connect_to_db(str(os.environ['db_host']), str(os.environ['db_name']), str(os.environ['db_user']), str(os.environ['db_password']))
print(read_db(full_response.choices[0].message.content,db_connection))
close_connection(db_connection)

Connection established
                                               0                        1
0                               COLDREX TABLETTA  paracetamol kombinációk
1                               COLDREX TABLETTA  paracetamol kombinációk
2          MEXALEN 125 MG VÉGBÉLKÚP CSECSEMŐKNEK              paracetamol
3       MEXALEN 250 MG VÉGBÉLKÚP KISGYERMEKEKNEK              paracetamol
4                   TEVA-MEXALEN 500 MG TABLETTA              paracetamol
5                   TEVA-MEXALEN 500 MG TABLETTA              paracetamol
6                              MIRALGIN TABLETTA  paracetamol kombinációk
7         NEO CITRAN BELSŐLEGES POR FELNŐTTEKNEK  paracetamol kombinációk
8                               SARIDON TABLETTA  paracetamol kombinációk
9  TALVOSILEN FORTE 500 MG/30 MG KEMÉNY KAPSZULA  paracetamol kombinációk


In [24]:
db_connection = connect_to_db(str(os.environ['db_host']), str(os.environ['db_name']), str(os.environ['db_user']), str(os.environ['db_password']))
print(read_db("SELECT OEP_FAN, OEP_FAB FROM GYOGYSZ WHERE OEP_NEV ILIKE '%syntroxine%' LIMIT '10'",db_connection))
close_connection(db_connection)

Connection established
   0  1
0  0  0
1  0  0
2  0  0
3  0  0
4  0  0
5  0  0
6  0  0
7  0  0
8  0  0
9  0  0
